<a href="https://colab.research.google.com/github/lauramenicacci/futurelab_ceres/blob/main/Extract_counts_citations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data collection from OECD Patent Database

Collect disaggregated patent raw data from OECD STI [Micro-data Lab](https://www.oecd.org/sti/intellectual-property-statistics-and-analysis.htmOECD/STI).

Files used:
* EPO_Inv_Reg from REGPAT: List of EPO inventors
* EPO_IPC from REGPAT: List of IPC classes and selected dates
* CPC_Classes from REGPAT: List of CPC classes related to the invention
* EPO_CIT_COUNTS from CITATIONS: Summary table
count 1 of backward & forward citations of EP patents
(Linked with EPO **Backward** Citations data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


 ## REGPAT - EPO

Collect patent data filed at the EPO.
 - ISO2 country code (Ctry_code)

Compute the count: inv_share x reg_share (according to OECD best practice)

Select the following vars before joining with citations:
* App_nbr EPO application number (13 character format “EPYYYYNNNNNNN”)
* Appln_id Surrogate key - applications in PATSTAT, Spring 2022
* Pub_nbr EPO patent publication number
* Ctry_code ISO 2 country code
* Reg_share Multiple allocation to a region - Share ≤ 1
* Inv_share Inventor’s share ≤ 1
* CPC_Class CPC codes
* Prio_year Priority year (first filing)
* App_year EPO filing year

In [ ]:
epo_inv = pd.read_csv("/content/drive/MyDrive/202308_EPO_Inv_reg.txt", encoding = "utf-8", sep = "|") # List of EPO inventors: contains appl_id

len(epo_inv)

11373131

In [ ]:
epo_inv_filt = epo_inv[['app_nbr', 'appln_id', 'pub_nbr', 'ctry_code', 'reg_share', 'inv_share']].drop_duplicates()

In [ ]:
# compute patent count multiplicating reg_share x inv_share

epo_inv_filt['count'] = epo_inv_filt.inv_share * epo_inv_filt.reg_share

In [ ]:
epo_inv_filt_counted = epo_inv_filt[['app_nbr', 'appln_id', 'pub_nbr', 'ctry_code', 'count']]

epo_inv_filt_counted.head()

,app_nbr,appln_id,pub_nbr,ctry_code,count
0,EP20000103094,1,1037159,FI,0.250000
4,EP20000107845,2,1024191,GB,0.166667
6,EP20000107845,2,1024191,US,0.166667
9,EP20000107845,2,1024191,AU,0.166667
10,EP20000202556,3,1070969,DE,0.333333


In [ ]:
cpc_class = pd.read_csv("/content/drive/MyDrive/202308_CPC_Classes.txt",  encoding = "utf-8", sep = "|") # list of CPC classes related to the invention

len(cpc_class)

61787610

In [ ]:
cpc_class.head()

,appln_id,CPC_Class
0,1,G06K 7/0013
1,1,G06K 7/0021
2,1,G06K 7/0043
3,1,G06K 7/0069
4,1,G06K 19/07739


In [ ]:
epo_ipc = pd.read_csv("/content/drive/MyDrive/202308_EPO_IPC.txt", encoding = "utf-8", sep = "|")

len(epo_ipc)

17410064

In [ ]:
epo_ipc.head()

,appln_id,prio_year,app_year,IPC
0,1,1999,2000,G06K007/00
1,1,1999,2000,G06K017/00
2,1,1999,2000,G06K019/077
3,1,1999,2000,H01R012/18
4,1,1999,2000,H04M001/02


In [ ]:
epo_cit_counts = pd.read_csv("/content/drive/MyDrive/202308_EPO_CIT_COUNTS.txt", encoding = "utf-8", sep = "|")

len(epo_cit_counts)

<ipython-input-9-d24ae74fff81>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  epo_cit_counts = pd.read_csv("/content/drive/MyDrive/202308_EPO_CIT_COUNTS.txt", encoding = "utf-8", sep = "|")


4127593

In [ ]:
# filter already now to have less data to merge

epo_cit_counts_filt = epo_cit_counts[['EP_Pub_nbr', 'EP_Pub_date','EP_Appln_id', 'EP_Pat_Cits', 'EP_NPL_Cits', 'WO_Pat_Cits', 'WO_NPL_Cits', 'Total_Pat_Cits', 'Total_Cits']].drop_duplicates()

In [ ]:
epo_cit_counts_filt.head()

,EP_Pub_nbr,EP_Pub_date,EP_Appln_id,EP_Pat_Cits,EP_NPL_Cits,WO_Pat_Cits,WO_NPL_Cits,Total_Pat_Cits,Total_Cits
0,EP0000001,1978-12-20,16428854,7,0,0,0,7,7
1,EP0000002,1978-12-20,16427091,2,1,0,0,2,3
2,EP0000003,1978-12-20,16429215,1,0,0,0,1,1
3,EP0000004,1978-12-20,16427120,4,0,0,0,4,4
4,EP0000005,1978-12-20,16427159,3,0,0,0,3,3


These are only **backward citations counts**\.

- EP_Pat_Cits = Number of patent citations made in European search [1]
- EP_NPL_Cits = Number of references to non patent literature (NPL) in European search [2]
- WO_Pat_Cits = Number of patent citations made in international search
[3]
- WO_NPL_Cits = Number of references to non patent literature (NPL) in international search [4]
- Total_Pat_Cits = Number of all patent citations made [1]+[3]= [5]
- Total_NPL_Cits = Number of all non patent literature citations made [2]+[4]= [6]
- Total_Cits = All citations made [5]+[6] = [7]

### Filter by Y02E class before merging

Unfortunately due to runtime limits.

In [ ]:
cpc_y02 =  cpc_class[cpc_class['CPC_Class'].str.startswith('Y02')]

In [ ]:
cpc_y02.head()

,appln_id,CPC_Class
176,14,Y02T 10/12
204,22,Y02D 30/70
258,29,Y02W 30/91
361,45,Y02W 30/91
510,67,Y02B 30/70


In [ ]:
len(cpc_y02)

1114763

### Join with list of IPC classes and CPC classes

In this way I get priority year.

In [ ]:
merged_ipc_cpc = pd.merge(epo_ipc, cpc_y02, how='left', on= 'appln_id') # merge epo patents with patent class dataset

In [ ]:
merged_ipc_cpc = merged_ipc_cpc.dropna()

In [ ]:
merged_ipc_cpc = merged_ipc_cpc[['appln_id', 'prio_year', 'app_year', 'CPC_Class']] # remove IPC codes as they don't contain the Y02 codes we are interested in

### Merge tech classes and year info with patent counts using appln_id

Merge the previous with EPO_Inv_reg = List of EPO inventors data that contains the count according to the inventor's address. Key = appln_id

In [ ]:
epo = pd.merge(epo_inv_filt_counted, merged_ipc_cpc, how='left', on= 'appln_id')

In [ ]:
epo = epo.dropna()

In [ ]:
epo = epo.drop_duplicates()

In [ ]:
epo

,app_nbr,appln_id,pub_nbr,ctry_code,count,prio_year,app_year,CPC_Class
15,EP20010907696,14,1409856,FR,0.500000,2000.0,2001.0,Y02T 10/12
32,EP20020258940,22,1324583,AU,1.000000,2001.0,2002.0,Y02D 30/70
42,EP20020739999,29,1399792,US,1.000000,2001.0,2002.0,Y02W 30/91
61,EP20030730922,45,1534646,NO,0.250000,2002.0,2003.0,Y02W 30/91
64,EP20030730922,45,1534646,AE,0.250000,2002.0,2003.0,Y02W 30/91
...,...,...,...,...,...,...,...,...
7038647,EP20220798047,582499466,4134154,US,0.333333,2021.0,2022.0,Y02A 50/20
7038648,EP20220798047,582499466,4134154,US,0.333333,2021.0,2022.0,Y02C 20/40
7038649,EP20220798047,582499466,4134154,US,0.333333,2021.0,2022.0,Y02P 20/151
7038673,EP20220209573,583404468,4122934,CN,0.166667,2021.0,2022.0,Y02E 10/549


### Merge with citations data

In [ ]:
epo_cit = pd.merge(epo, epo_cit_counts_filt, how='left', left_on = 'appln_id', right_on = 'EP_Appln_id').dropna().drop_duplicates()

In [ ]:
epo_cit

,app_nbr,appln_id,pub_nbr,ctry_code,count,prio_year,app_year,CPC_Class,EP_Pub_nbr,EP_Pub_date,EP_Appln_id,EP_Pat_Cits,EP_NPL_Cits,WO_Pat_Cits,WO_NPL_Cits,Total_Pat_Cits,Total_Cits
0,EP20010907696,14,1409856,FR,0.500000,2000.0,2001.0,Y02T 10/12,EP1409856,2004-04-21,14,0,0,7,0,7,7
1,EP20020258940,22,1324583,AU,1.000000,2001.0,2002.0,Y02D 30/70,EP1324583,2003-07-02,22,5,0,0,0,5,5
2,EP20020739999,29,1399792,US,1.000000,2001.0,2002.0,Y02W 30/91,EP1399792,2004-03-24,29,1,0,4,0,5,5
3,EP20030730922,45,1534646,NO,0.250000,2002.0,2003.0,Y02W 30/91,EP1534646,2005-06-01,45,0,0,5,0,5,5
4,EP20030730922,45,1534646,AE,0.250000,2002.0,2003.0,Y02W 30/91,EP1534646,2005-06-01,45,0,0,5,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552435,EP20220798047,582499466,4134154,US,0.333333,2021.0,2022.0,Y02A 50/20,EP4134154,2023-02-15,582499466,0,0,10,0,10,10
552436,EP20220798047,582499466,4134154,US,0.333333,2021.0,2022.0,Y02C 20/40,EP4134154,2023-02-15,582499466,0,0,10,0,10,10
552437,EP20220798047,582499466,4134154,US,0.333333,2021.0,2022.0,Y02P 20/151,EP4134154,2023-02-15,582499466,0,0,10,0,10,10
552438,EP20220209573,583404468,4122934,CN,0.166667,2021.0,2022.0,Y02E 10/549,EP4122934,2023-01-25,583404468,2,0,0,0,2,2


### Save

In [ ]:
epo_cit.to_csv('epo_counts_backcits.csv', sep = ',', index = False) # save